# Backend

## Crime Analysis Dashboard App

In [1]:
#Import Libraries
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.ops import triangulate

## Get Police Force Table

In [ ]:

def get_forces():

    """ 
    
    Defines the function to call the API and retrieve the police forces data table.
    
    Input: None.
    
    Output: Dataframe containing all the police forece ids and names.
    
    """

    # Defines the connection to the API
    url = "https://data.police.uk/api/forces"
    response = requests.get(url)

    # Raises exception if connection fails
    if response.status_code != 200:                                 
        raise Exception(f"API error: {response.status_code}")           
    
    # Converts responce to json
    data = response.json()

    # Converts json to dataframe
    df = pd.DataFrame(data)

    # Rename ID and name columns
    df.rename(columns={"id": "police_force_id", "name": "police_force_name"}, inplace=True)

    # Returns dataframe
    return df

# Defines the police force dataframe
df_forces = get_forces()

print(df_forces.head())

     police_force_id               police_force_name
0  avon-and-somerset  Avon and Somerset Constabulary
1       bedfordshire             Bedfordshire Police
2     cambridgeshire     Cambridgeshire Constabulary
3           cheshire           Cheshire Constabulary
4     city-of-london           City of London Police


## Get Neighbourhood Table

In [3]:

# INPUT FROM USER
police_force_ids = ["bedfordshire", "cambridgeshire"]  

def get_neighbourhood(id):

    """

    Defines the function to call the API and retrieve the neighbourhood data table.

    Input: Police force ID.

    Output: Dataframe containing the neighbouthood id, name and which police force id it belongs to.

    """

    # Defines the connection to the API
    url = f"https://data.police.uk/api/{id}/neighbourhoods"     # Defines the API url 
    response = requests.get(url)                                # Defines the responce after we 'get' the url
    
    # Raises exception if connection fails
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code}")
    
    # Converts responce to json 
    data = response.json()

    # Converts json to dataframe
    df = pd.DataFrame(data)

    # Adds police force ID
    df['police_force_id'] = id

    # Renames the ID and name columns
    df.rename(columns={"id": "neighbourhood_id", "name": "neighbourhood_name"}, inplace=True)

    # Returns dataframe
    return df

# Empty list of neighbourhood dataframes 
df_neighbourhoods_list =[]

# Appends each neighbourhood dataframe to the list
for id in police_force_ids:
    df_neighbourhoods_list.append(get_neighbourhood(id))

# Unions the list of dataframes into one
df_neighbourhoods = pd.concat(df_neighbourhoods_list)

print(df_neighbourhoods.sample(5))

            neighbourhood_id           neighbourhood_name police_force_id
2                        BD5            Riseley, Wyboston    bedfordshire
3         CamCity_South_City              Cambridge South  cambridgeshire
8   Peterborough_City_Centre     Peterborough City Centre  cambridgeshire
7                        CB5     Houghton Regis and Rural    bedfordshire
16                       NU4  Putnoe, Goldington, Newnham    bedfordshire


## Get Neighbourhood Boundaries Table

In [ ]:

def get_neighbourhood_boundaries(neighbourhood_id):

    """

    Defines the function to call the API and retrieve the neighbourhood boundaries data table.

    Input: Neighbourhood ID.

    Output: Corresponding boundary polygon string.
    
    """

    # Finds the input's corresponding police force ID 
    police_force_id = df_neighbourhoods.loc[df_neighbourhoods["neighbourhood_id"] == neighbourhood_id, "police_force_id"].iloc[0]

    # Defines the connection to the API
    url = f"https://data.police.uk/api/{police_force_id}/{neighbourhood_id}/boundary"
    response = requests.get(url)
    
    # Raises exception if connection fails
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code}")
    
    # Convert responce to json
    data = response.json()

    # Convert to polygon string format
    polygon_str = ":".join(
        f"{float(item['latitude'])},{float(item['longitude'])}"
        for item in data
    )

    # Returns polygon
    return polygon_str

# Makes a copy of the neighbourhoods dataframe
df_neighbourhood_boundaries = df_neighbourhoods.copy()

# Initiates empty boundary list 
boundary_list = []

# Loops through all neighbourhood ID's and appends the boundary polygon to the list
for neighbourhood_id in df_neighbourhood_boundaries["neighbourhood_id"]:
    boundary_list.append(get_neighbourhood_boundaries(neighbourhood_id))

# Defines the boundary list as a new column in the dataframe
df_neighbourhood_boundaries["neighbourhood_boundary"] = boundary_list

print(df_neighbourhood_boundaries.head())

  neighbourhood_id                         neighbourhood_name police_force_id  \
0              BD2                   Bromham, Oakley, Wootton    bedfordshire   
1              BD3           Wilstead, Shortstown, Willington    bedfordshire   
2              BD5                          Riseley, Wyboston    bedfordshire   
3              CB1  Flitwick, Ampthill, Marston and Cranfield    bedfordshire   
4              CB2    Biggleswade, Sandy, Potton and Shefford    bedfordshire   

                              neighbourhood_boundary  
0  52.200075358,-0.548369711:52.200155837,-0.5484...  
1  52.201040284,-0.437150426:52.20106348,-0.43761...  
2  52.322913907,-0.465104578:52.322953267,-0.4653...  
3  52.110251611,-0.591229997:52.11069521,-0.59140...  
4  52.190902794,-0.287653471:52.190925829,-0.2891...  


## Get Street-Level Crimes Table

In [14]:

def get_street_level_crimes(poly_str):

    """
    
    Defines the function to call the API and retrieve the neighbourhood boundaries data table.

    Input: A polygon string defining the area to get street-level crimes for.

    Output: Street-level crime dataframe for the neighbourhood.

    """

    # Defines the connection to the API
    url = f"https://data.police.uk/api/crimes-street/all-crime?poly={poly_str}"
    response = requests.get(url)
    
    # Raises exception if connection fails
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code}")
    
    # Convert responce to json
    data = response.json()

    # Converts json to dataframe
    df = pd.DataFrame(data)

    # Returns dataframe
    return df



def load_polygon_from_kml(filepath):

    """

    A function to load a polygon from a KML file.

    Input: Filepath to the KML file.

    Output: Shapely polygon object.

    """

    gdf = gpd.read_file(filepath, driver="LIBKML")

    # Many KMLs contain a single feature
    polygon = gdf.geometry.iloc[0]

    return polygon



def triangulate_polygon(polygon):

    """

    A function to triangulate a polygon into smaller triangles.

    Input: Shapely polygon object.

    Output: List of Shapely triangle objects.

    """

    triangles = triangulate(polygon)

    # Keep only triangles fully inside the polygon
    triangles = [t for t in triangles if polygon.contains(t.centroid)]

    return triangles



def triangle_to_poly_string(triangle):
    
    """

    A function to convert a triangle to a polygon string.

    Input: Shapely triangle object.

    Output: Polygon string.

    """

    coords = list(triangle.exterior.coords)[:-1]  # remove repeated closing point

    # convert (lng, lat) → (lat, lng)
    return ":".join(
        f"{coord[1]},{coord[0]}"
        for coord in coords
    )



def simplify_polygon(polygon, tolerance=0.0005):

    """

    Reduce polygon complexity while preserving shape.
    Tolerance ≈ 50m at UK latitudes.

    Input: Shapely polygon object, tolerance value.

    Output: Simplified Shapely polygon object.

    """

    return polygon.simplify(tolerance, preserve_topology=True)



def process_kml_file_to_dataframe(kml_path):

    """

    A function to process a KML file and retrieve street-level crime data for the area.

    Input: Filepath to the KML file.

    Output: Street-level crime dataframe for the area.

    """

    # Loads polygon
    polygon = load_polygon_from_kml(kml_path)

    # Simplifies polygon
    polygon = simplify_polygon(polygon)

    # Triangulates polygon
    triangles = triangulate_polygon(polygon)

    # Initialises a list to collect each triangle's dataframe
    all_dfs = []

    # Gets dataframa for each triangle
    for tri in triangles:
        # Converts the triangles to lists of coordiante strings
        poly_str = triangle_to_poly_string(tri)

        # Calls the API and gets the dataframe
        df = get_street_level_crimes(poly_str)

        # Combine dataframes
        all_dfs.append(df)

    # Checks that data has been collected, returns blank dataframe if no data is collected
    if len(all_dfs) == 0:
        return pd.DataFrame()

    # Combines all dataframes
    final_df = pd.concat(all_dfs, ignore_index=True)

    # Returns combined dataframe
    return final_df

df_crimes = process_kml_file_to_dataframe("C:/Users/benco/Downloads/force_kmls/force kmls/hertfordshire.kml")
print(df_crimes.head())



                category location_type  \
0          violent-crime         Force   
1  criminal-damage-arson         Force   
2          vehicle-crime         Force   
3  anti-social-behaviour         Force   
4  anti-social-behaviour         Force   

                                            location context  \
0  {'latitude': '52.062879', 'street': {'id': 194...           
1  {'latitude': '51.684363', 'street': {'id': 194...           
2  {'latitude': '51.684363', 'street': {'id': 194...           
3  {'latitude': '51.685564', 'street': {'id': 194...           
4  {'latitude': '51.684728', 'street': {'id': 194...           

                                      outcome_status  \
0  {'category': 'Under investigation', 'date': '2...   
1  {'category': 'Investigation complete; no suspe...   
2  {'category': 'Investigation complete; no suspe...   
3                                               None   
4                                               None   

                         

In [15]:
df_crimes.shape

(8992, 9)